# Criptología
## Criptosistema de Merkle-Hellman.
### Ángel Ríos San Nicolás

Implementamos una versión del criptosistema de Merkle-Hellman, que es un método basado en el problema de la mochila. El criptosistema funciona como sigue:

Consideramos un $n\in\mathbb{N}$ y una sucesión supercreciente $(b_1,\ldots,b_n)\in\mathbb{N}^n$, es decir, tal que para todo $i\in\{1,\ldots, n\}$ se cumple
$$b_i = \sum_{k=1}^{i-1}b_k.$$

Tomamos $U,V\in\mathbb{N}$ de manera aleatoria tales que son coprimos y $U>\sum_{k=1}^n b_k$ y también $U>V$. 

Definimos $(a_1,\ldots,a_n)\in\mathbb{N}^n$ dado por $$a_i = b_iV\mod U$$ para todo $k\in\{1,\ldots, n\}$.

La clave pública es $(a_1,\ldots, a_n)$ y la clave privada es $((b_1,\ldots, b_n), U, V)$.

Vamos a implementar una manera de generar pseudoaleatoriamente las claves a partir de un $n$ fijo. En particular, tenemos que diseñar una manera de generar sucesiones supercrecientes de manera pseudoaleatoria para lo que hay diversas maneras de hacerlo de entre las que proponemos una.

In [1]:
# Generación de las claves pública y privada.
def claves(n):
    BS = [randint(2,2*n)]  # Guardará la sucesión supercreciente de la clave privada.
    for _ in range(n):
        s = sum(BS)  # Suma de los anteriores.
        c = floor(mean(BS)/2)  # Parámetro para acotar la generación de pseudoaleatorios.
        # c = max(BS)  # Alternativa.
        BS.append(s + randint(1,c))  # Añadimos un positivo pseudoaleatorio y acotado.
    sumBS = sum(BS)
    V = randint(sumBS, 2 * sumBS + c)  # Tomamos V > sumBS pseudoaleatorio y acotado.
    GCD = 0
    iters = 0
    while GCD != 1 or iters == 10000:
        U = randint(V, 2 * V)
        GCD = gcd(U,V)
        iters += 1
    if iters == 10000:  # Con probabilidad prácticamente nula.
        U = V+1  # gcd(V, V+1) = 1
    AS = [(BS[i] * V) % U for i in range(len(BS))]  # Construimos la clave pública a=bV mod U.
    return AS, BS, U, V
[AS, BS, U, V] = claves(1000)

Notamos que hay decisiones arbitarias. El parámetro $c$ que se utiliza para acotar los números pseudoaleatorios que generamos se podría haber tomado fijo para todas las iteraciones, generado a su vez pseudoaleatoriamente de alguna manera o dependiente de la sucesión supercreciente que se va generando pero con distinta definición.  Podríamos tomar cotas más altas, por ejemplo, $c=\max(BS)$ y los cálculos siguen siendo muy rápidos. Podemos hacer las mismas observaciones sobre la elección de cotas en las definiciones de $V$ y $U$. Otra observación es que la condición de parada en iteraciones se da con probabilidad prácticamente nula, la añadimos solo para que sea un algoritmo y en ese caso tomamos $U=V+1$ que es coprimo con $V$.

Implementamos el cifrado que dado un mensaje $0\leq m\leq 2^n$ considera la representación binaria $m=[m_1,\ldots m_n]$, es decir, 
$m = \sum_{i=1}^nm_i2^{i-1}$ y
para la que el cifrado es $$y=\sum_{i=1}^n m_ia_i$$ siendo $(a_1,\ldots, a_n)$ la clave pública.

In [2]:
# CIFRADO
def cifra(mensaje, AS):
    c = bin(mensaje)[2:]  # Representación binaria de c.
    c = "0" * (len(AS) - len(c)) + c # Completamos la representación binaria con ceros a izda.
    a = []
    for _ in c:
        a.append(int(_))
    return sum([AS[_] * a[_] for _ in range(len(AS))])

Veamos un ejemplo de cifrado para un mensaje pseudoaleatorio $0\leq m < 2^{1000}$.

In [3]:
mensaje = randint(0,2^1000)
y = cifra(mensaje, AS)
print("Mensaje: ", mensaje)
print("Mensaje cifrado: ", y)

Mensaje:  9510546014108545760587043950280028273724493180558023749004520421695107168130660670585983616775371328858359482534718496733573806815277179270231323250405990520534357334347650757056486631703776087631216350572579657979534918594289654316993022790511455444866646928232548861936667371024131064806478927596664
Mensaje cifrado:  17957036692147165165774379466007864789154405296682079012467674830951771959740783243806865587468310697137532550663726831767525908580343900465987821069585625334735523570415273555169202796773001184428043908995511781726995406696072882375851353955779158365625511711308503840163998010005751013869612652833031735452


Implementamos el descifrado que dado el mensaje cifrado $y$ y la clave privada $(b_1,\ldots,b_n)$, si el mensaje descifrado es $m=[m_1,\ldots, m_n]$ en binario, entonces se puede calcular de forma eficiente $m$ en representación binaria resolviendo el problema de la mochila $S=\sum_{i=1}^n m_ib_i=yV^{-1}\mod U$ porque la sucesión $(b_1,\ldots,b_n)$ es supercreciente. La manera de hacerlo es considerar el $b_i$ más grande, en este caso $b_n$ porque están ordenados y comproban si satura o no la mochila. Tomamos el primer elemento $b_n$, si es $b_n>S$, entonces $m_n=0$. Si no, $m_n=1$. Restando $m_n$ al total, tenemos el problema de la mochila $S-m_n = \sum_{i=1}^{n-1}m_ib_i$ y repetimos el mismo proceso hasta que los elementos escogidos sumen $S$.

In [4]:
# DESCIFRADO
def creciente_mochila(lista, S):  # Descifra lista a binario con tamaño de mochila S.
    n = len(lista)
    x = [0 for i in range(n)]  # Guardará el mensaje descifrado.
    aux = S
    for i in range(n):
        if aux >= lista[n-1-i]:
            x[n-1-i] = 1
            aux = aux - lista[n-1-i]
    if aux == 0:
        return x
    return -1

def listbin_a_dec(x):  # Convierte una lista binaria a decimal.
    x = [str(_) for _ in x]
    cadena = ""
    for _ in x:
        cadena += _
    return int(cadena, base = 2)

def descifra(y, BS, U, V):  # Descifra el mensaje y a decimal con la clave privada (BS, U, V).
    S = (y * power_mod(V, -1, U)) % U  # Tamaño de la mochila.
    des = creciente_mochila(BS, S)
    return listbin_a_dec(des)

Comprobamos que podemos descifrar el mensaje que generamos y ciframos previamente.

In [5]:
x = descifra(y, BS, U, V)
print("Mensaje descifrado: ", x)
print(mensaje == x)  # Observamos que el mensaje descifrado coincide con el mensaje que ciframos.

Mensaje descifrado:  9510546014108545760587043950280028273724493180558023749004520421695107168130660670585983616775371328858359482534718496733573806815277179270231323250405990520534357334347650757056486631703776087631216350572579657979534918594289654316993022790511455444866646928232548861936667371024131064806478927596664
True


Vamos a dar una versión del criptosistema para cifrar y descifrar texto. Para ello, vemos primero un ejemplo en el que codificamos y decodificamos entre texto y binario. Esto nos servirá para aprovechar el método anterior.

In [6]:
m = "Esta frase es mentira".encode()
num = int.from_bytes(m, "big")
en_binario = bin(num)[2:]
en_binario

'10001010111001101110100011000010010000001100110011100100110000101110011011001010010000001100101011100110010000001101101011001010110111001110100011010010111001001100001'

In [7]:
binnum = int(en_binario, 2)
orden = binnum.bit_length() + 7 // 8
mensaje = binnum.to_bytes(orden, "big").decode()
mensaje = mensaje.replace("\x00","")
mensaje

'Esta frase es mentira'

Implementamos el método de cifrado exactamente de la misma forma salvo que transformamos el mensaje a decimal previamente para aprovechar el anterior.

In [8]:
# CIFRADO
def cifra_txt(mensaje, AS):
    mensaje = mensaje.encode()
    num = int.from_bytes(mensaje, "big")
    en_binario = bin(num)[2:]
    y = cifra(int(str(en_binario), base = 2), AS)  # Aprovechamos el cifrado decimal.
    return y  # No lo podemos pasar a texto porque en general no codifica ninguno.

Vamos a ver un ejemplo de cifrado de "Criptosistema de Merkle-Hellman''.

In [9]:
mensaje = "Criptosistema de Merkle-Hellman"
y = cifra_txt(mensaje, AS)
print("Mensaje: ", mensaje)
print("Mensaje cifrado: ", y)

Mensaje:  Criptosistema de Merkle-Hellman
Mensaje cifrado:  4322638271625426403826762043246011935730186072100492394042292427908234117848329613867835082090487000295893440404858382368263575097361004543809407102569003189388320849438057260627946529529113993842236719950717305315230974887123890992906969429762900060651391917268268103699294072930253426887517209477997478854


El descifrado se hace de manera análoga, aprovechamos el descifrado en decimal y transformamos el resultado a una cadena de caracteres.

In [10]:
# DESCIFRADO
def descifra_txt(y, BS, U, V):
    x = descifra(y, BS, U, V)  # Aprovechamos el descifrado decimal.
    orden = x.bit_length() + 7 // 8
    mensaje = x.to_bytes(orden, "big").decode()
    mensaje = mensaje.replace("\x00","")
    return mensaje

Comprobamos que podemos recuperar el mensaje que ciframos previamente.

In [11]:
x = descifra_txt(y, BS, U, V)
print("Mensaje descifrado: ", x)
mensaje == x  # El texto descifrado coincide con el cifrado.

Mensaje descifrado:  Criptosistema de Merkle-Hellman


True